Building A Chatbot!

here we are to design and implement an LLM_powered chatbot. This Chatbot will be able to have a conversation and remeber previous interactions 

Note : This chatbot will only use the language model to have the conversation there are several other related concepts that you may be looking for : 

1.Conversational RAG : Enable a chatbot experience over external source of data 

2.Agents : Build a chatbots that can take actions




In [17]:
import os 
from dotenv import load_dotenv
load_dotenv() ## loading all the nevironment variables 

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key


'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'

In [18]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026F26E41490>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026F26E363F0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [19]:
from langchain.schema import HumanMessage

# Invoke the model with a human message
response = model.invoke([HumanMessage(content="hi, my name is Raman Srivastava")])
print(response)

content="Hello Raman Srivastava, it's nice to meet you! How can I help you today? 😄  \n\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 16, 'total_tokens': 41, 'completion_time': 0.045454545, 'prompt_time': 0.002030455, 'queue_time': 0.24958269700000002, 'total_time': 0.047485}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-391ac6a1-0df2-47f4-9b3e-42e14720f3c1-0' usage_metadata={'input_tokens': 16, 'output_tokens': 25, 'total_tokens': 41}


In [20]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="hi, my name is Raman Srivastava"),
        AIMessage(content="Hello Raman! How can I assist you today?"),
        HumanMessage(content="Hey What is my name?")
    ]
)



AIMessage(content='Your name is Raman Srivastava. 😊  I remember that from our first interaction.\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 42, 'total_tokens': 61, 'completion_time': 0.034545455, 'prompt_time': 0.003034153, 'queue_time': 0.24736622899999997, 'total_time': 0.037579608}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-34ececc8-dd78-4244-a935-f0f665252e68-0', usage_metadata={'input_tokens': 42, 'output_tokens': 19, 'total_tokens': 61})

### Message history 

We can usea Message history class to wrap out model and make it statful; . This will keep record of inputs and outputs of the model, and store them in data store. Future interaction will htne load those messages and pass them into chain as part of the input.

Lets see how to use this!

In [21]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store={}

def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)


In [22]:
config={"configurable":{"session_id":"chat1"}}

In [ ]:
# Ensure config is defined

response = with_message_history.invoke(
    [HumanMessage(content="Hi , my name is Raman Srivastava and i am ai engineer")],
    config=config,
)

In [27]:
response.content 

"Hi Raman Srivastava!  It's great to meet you.  \n\nBeing an AI engineer yourself, what are you working on these days? Anything exciting? 😊  \n\n"

In [28]:
with_message_history.invoke(
    [HumanMessage(content="Hi , what is my name")],
    config=config,
)

AIMessage(content='Your name is Raman Srivastava, you told me earlier!  \n\nHow can I help you today, Raman? 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 247, 'total_tokens': 274, 'completion_time': 0.049090909, 'prompt_time': 0.009400449, 'queue_time': 0.24840935699999997, 'total_time': 0.058491358}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0f40fda7-cc16-4222-a863-bf2a2040b460-0', usage_metadata={'input_tokens': 247, 'output_tokens': 27, 'total_tokens': 274})

In [30]:
## change the config to a different session id and see the result

config={"configurable":{"session_id":"chat2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi , What is my name")],
    config=config,
)

In [31]:
response.content

"I'm just a friendly AI, and I don't have access to any information about you, including your name.  \n\nIf you'd like to tell me your name, I'd love to know! 😊  \n"

In [32]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi , my namae is raman srivastava")],
    config=config,
)

In [33]:
response.content 

"Hello Raman Srivastava! It's nice to meet you. 😊  \n\nHow can I help you today?  \n\n"

In [34]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi , what is my name?")],
    config=config,
)

In [35]:
response.content

'You told me your name is Raman Srivastava!  😊 \n\nIs there anything else I can help you with?\n'

Prompt  Templates :

prompt template help to turn raw user information into a format that llm can work with in this case the raw user input is just a message ,which we are passing to the LLM . lets now make a that but more ccomplicated First, lets add in a system message with some custom instructions(but still taking message as input.). next we will add in more inputs besided the mmessages

In [40]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder 
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "You are a helpful assistant answer all the question to best of your ability.",
        ),
    MessagesPlaceholder(variable_name="messages"),
    ]
)

chain=prompt|model

In [43]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is raman")]})

AIMessage(content="Hello Raman, it's nice to meet you! 😊\n\nI'm happy to help answer any questions you have. Just ask away!  \n\nWhat can I do for you today? \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 30, 'total_tokens': 74, 'completion_time': 0.08, 'prompt_time': 0.005096349, 'queue_time': 0.906158664, 'total_time': 0.085096349}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e6711e0d-3495-4dcb-a34f-9c2d7f58ca6d-0', usage_metadata={'input_tokens': 30, 'output_tokens': 44, 'total_tokens': 74})

In [44]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)


In [48]:
config  = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content = "Hi My name is Raman")],
    config=config
)

response.content


"Hi Raman, it's nice to meet you! 😊  \n\nWhat can I do for you today?\n"

In [52]:
## add more complexity 

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a helpful assistant. answer al the questions to best of your ablity in {language}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model 

In [54]:
response = chain.invoke({"messages":[HumanMessage(content="hi my name is Raman ")],"language": "Hindi"})
response.content

'नमस्ते Raman! \n\nमुझे आपकी मदद करने में खुशी हो रही है। आप क्या जानना चाहते हैं? \n\nमुझे आपके सवालों के जवाब देने और आपकी मदद करने के लिए तैयार रहूँगा।  \n\n'

Lets now wrap this more complicated chain in messages history class.
this time,because there were multiple keys input in the input, we need to specify the correct key to use to save the chat history

In [68]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [70]:
config = {"configurable": {"session_id": "chat4"}}
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="hi, my name is raman ?")],
        "language": "Hindi"
    },
    config=config
)

print(response.content)

नमस्ते Raman, 

(Hello Raman,)

मुझे आपसे मिलकर खुशी हो रही है! 

(It's nice to meet you!) 

क्या मैं आपकी कोई मदद कर सकता हूँ?

(Can I help you with anything?) 



In [80]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi what is the my name")], "language": "Hindi"},
    config=config,
)

In [81]:
response.content 

'आपका नाम Raman है। \n\n(Your name is Raman.) \n'

Managing te Conversation History 

One import concept to understand when building chatbots how to manage conversation history. If left unmanaged, the list will grow unbounded and protentially overflow the context window of the LLM. therefore it is important to add a step that limitts the size of the messages you are passing in. 

'trim_messages' = it helps to reduce the size of the messages you are passing in. uses partial messages.

In [88]:
from langchain_core.messages import SystemMessage,trim_messages 
trimmer = trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you are a good assistant"),
    AIMessage(content="Hello, my name is raman?"),
    HumanMessage(content="What is your name?"),
    AIMessage(content="My name is raman"),
    HumanMessage(content="What is your age?"),
    AIMessage(content="I am 23 years old"),
    HumanMessage(content="What is your hobbies?"),
    AIMessage(content="I love to play games and watch movies"),
    HumanMessage(content="What is your favorite game?"),
    AIMessage(content="My favorite game is chess"),
    HumanMessage(content="What is your favorite movie?"),
    AIMessage(content="My favorite movie is Avengers")
]

trimmer.invoke(messages)

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your age?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I am 23 years old', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your hobbies?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I love to play games and watch movies', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your favorite game?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='My favorite game is chess', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your favorite movie?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='My favorite movie is Avengers', additional_kwargs={}, response_metadata={})]

In [94]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)  # preprocess messages
    | prompt  # injects messages + language into template
    | model   # calls the LLM
)


response = chain.invoke({
    "messages": messages + [HumanMessage(content="What is your hobbies?")],
    "language": "English"
})


response.content 

"As an AI, I don't experience emotions or have hobbies in the way humans do. I can process information and respond to your requests, which is kind of like a hobby for me!  \n\nDo you have any hobbies you'd like to tell me about?\n"

In [95]:
## Lets Wrap this in the Message History

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "chat5"}}

In [96]:
response = with_message_history.invoke(
    {"messages": messages +  [HumanMessage(content="hi what is the my name")], "language": "Hindi"},
    config=config,
)

response.content

"मुझे नहीं पता कि आपका नाम क्या है। क्या आप मुझे बता सकते हैं?  \n\n(I don't know your name. Can you tell me?) \n"